# Collaboration and Competition

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the third project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [1]:
from unityagents import UnityEnvironment
import numpy as np

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Tennis.app"`
- **Windows** (x86): `"path/to/Tennis_Windows_x86/Tennis.exe"`
- **Windows** (x86_64): `"path/to/Tennis_Windows_x86_64/Tennis.exe"`
- **Linux** (x86): `"path/to/Tennis_Linux/Tennis.x86"`
- **Linux** (x86_64): `"path/to/Tennis_Linux/Tennis.x86_64"`
- **Linux** (x86, headless): `"path/to/Tennis_Linux_NoVis/Tennis.x86"`
- **Linux** (x86_64, headless): `"path/to/Tennis_Linux_NoVis/Tennis.x86_64"`

For instance, if you are using a Mac, then you downloaded `Tennis.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Tennis.app")
```

In [2]:
class EnvironmentWrapper():
    def __init__(self, file_name):
        self.env = UnityEnvironment(file_name=file_name, worker_id=1)
        self.brain_name = self.env.brain_names[0]
        self.brain = self.env.brains[self.brain_name]
        self.action_size = self.brain.vector_action_space_size
        
        self.env_info = self.env.reset(train_mode=False)[self.brain_name]
        states = self.env_info.vector_observations
        self.state_size = states.shape[1]
        
    def reset(self, train_mode=False):
        self.env_info = self.env.reset(train_mode=train_mode)[self.brain_name]
        state = self.env_info.vector_observations
        
        return state
        
    def close(self):
        self.env.close()
        
    def step(self, actions):
        self.env_info = self.env.step(actions)[self.brain_name]       
        next_state = self.env_info.vector_observations   
        reward = self.env_info.rewards
        #reward = [x * 10.0 for x in reward]
        done = self.env_info.local_done                
        return next_state, reward, done
    
    
env = EnvironmentWrapper(file_name="tennis/tennis.exe")

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: TennisBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 8
        Number of stacked Vector Observation: 3
        Vector Action space type: continuous
        Vector Action space size (per agent): 2
        Vector Action descriptions: , 


In [4]:
##env = UnityEnvironment(file_name="tennis/tennis.exe")

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: TennisBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 8
        Number of stacked Vector Observation: 3
        Vector Action space type: continuous
        Vector Action space size (per agent): 2
        Vector Action descriptions: , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [3]:
# get the default brain
brain_name = env.env.brain_names[0]
brain = env.env.brains[brain_name]

### 2. Examine the State and Action Spaces

In this environment, two agents control rackets to bounce a ball over a net. If an agent hits the ball over the net, it receives a reward of +0.1.  If an agent lets a ball hit the ground or hits the ball out of bounds, it receives a reward of -0.01.  Thus, the goal of each agent is to keep the ball in play.

The observation space consists of 8 variables corresponding to the position and velocity of the ball and racket. Two continuous actions are available, corresponding to movement toward (or away from) the net, and jumping. 

Run the code cell below to print some information about the environment.

In [5]:
# reset the environment
env.reset(train_mode=True)

env_info = env.env_info

# number of agents 
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 2
Size of each action: 2
There are 2 agents. Each observes a state with length: 24
The state for the first agent looks like: [ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.         -6.69487906 -1.5
 -0.          0.          6.83172083  5.98822832 -0.          0.        ]


### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agents and receive feedback from the environment.

Once this cell is executed, you will watch the agents' performance, if they select actions at random with each time step.  A window should pop up that allows you to observe the agents.

Of course, as part of the project, you'll have to change the code so that the agents are able to use their experiences to gradually choose better actions when interacting with the environment!

In [12]:
for i in range(1, 5):                                      # play game for 5 episodes
    env.reset(train_mode=False)
    env_info = env.env_info     # reset the environment    
    states = env_info.vector_observations                  # get the current state (for each agent)
    scores = np.zeros(num_agents)                          # initialize the score (for each agent)
    while True:
        actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
        actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
        next_states, rewards, dones = env.step(actions)                    # get next state (for each agent)
        scores += rewards                                  # update the score (for each agent)
        states = next_states                               # roll over states to next time step
        if np.any(dones):                                  # exit loop if episode finished
            break
    print('Score (max over agents) from episode {}: {}'.format(i, np.max(scores)))

Score (max over agents) from episode 1: 0.0
Score (max over agents) from episode 2: 0.20000000298023224
Score (max over agents) from episode 3: 0.0
Score (max over agents) from episode 4: 0.0


In [13]:

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [75]:
class Actor(nn.Module):
    """Actor (Policy) Model."""

    def __init__(self, state_size, action_size, seed):
        """Initialize parameters and build model.
        Params
        ======
            state_size (int): Dimension of each state
            action_size (int): Dimension of each action
            seed (int): Random seed
        """
        super(Actor, self).__init__()
        self.seed = torch.manual_seed(seed)
        self.action_size = action_size
        self.state_size =  state_size
        
        self.fc1 = nn.Linear(self.state_size, 64)
        self.batch_norm1 =  torch.nn.BatchNorm1d(64)
        self.fc2 = nn.Linear(64, 32)
        self.batch_norm2 =  torch.nn.BatchNorm1d(32)
        self.fc3 = nn.Linear(32, self.action_size)
        
    def forward(self, state):
        """Build a network that maps state -> action values."""
        # Layer 1
        x = self.fc1(state)
        #x = self.batch_norm1(x)
        x = F.tanh(x)
        #x = F.dropout(x, p=0.2, training=self.training)
        
        # Layer 2
        x = self.fc2(x)
        #x = self.batch_norm2(x)
        x = F.tanh(x)
        #x = F.dropout(x, p=0.2, training=self.training)
        
        # Layer 3
        x = F.tanh(self.fc3(x))     
        
        return x
    
class Critic(nn.Module):
    """Critic (Q-Network) Model."""

    def __init__(self, state_size, action_size, seed):
        """Initialize parameters and build model.
        Params
        ======
            state_size (int): Dimension of each state
            action_size (int): Dimension of each action
            seed (int): Random seed
        """
        super(Critic, self).__init__()
        self.seed = torch.manual_seed(seed)
        self.action_size = action_size
        self.state_size =  state_size
        
        self.fc1s = nn.Linear(self.state_size, 64)
        self.batch_norm1 =  torch.nn.BatchNorm1d(64)
        self.fc2 = nn.Linear(64 + self.action_size, 64)
        self.fc3 = nn.Linear(64, 1)
                
    def forward(self, state, action):
        """Build a network that maps state -> action values."""
        # State feature layer
        xs = self.fc1s(state)
        xs = self.batch_norm1(xs)
        xs = F.relu(xs)
        
        # State freatures + action values
        x =  torch.cat((xs, action), dim=1)
        
        # Layer 2
        x = F.relu(self.fc2(x))
        x = F.dropout(x, p=0.2, training=self.training)
        
        # Layer 3
        x = self.fc3(x)     
         
        return x

In [76]:
class ReplayBuffer:
    """Fixed-size buffer to store experience tuples."""

    def __init__(self, action_size, buffer_size, batch_size, seed):
        """Initialize a ReplayBuffer object.

        Params
        ======
            action_size (int): dimension of each action
            buffer_size (int): maximum size of buffer
            batch_size (int): size of each training batch
            seed (int): random seed
        """
        self.action_size = action_size
        self.memory = deque(maxlen=buffer_size)  
        self.batch_size = batch_size
        self.experience = namedtuple("Experience", field_names=["state", "action", "reward", "next_state", "done"])
        self.seed = random.seed(seed)
    
    def add(self, state, action, reward, next_state, done):
        """Add a new experience to memory."""
        e = self.experience(state, action, reward, next_state, done)
        self.memory.append(e)
    
    def sample(self):
        """Randomly sample a batch of experiences from memory."""
        experiences = random.sample(self.memory, k=self.batch_size)

        states = torch.from_numpy(np.vstack([e.state for e in experiences if e is not None])).float().to(device)
        actions = torch.from_numpy(np.vstack([e.action for e in experiences if e is not None])).float().to(device)
        rewards = torch.from_numpy(np.vstack([e.reward for e in experiences if e is not None])).float().to(device)
        next_states = torch.from_numpy(np.vstack([e.next_state for e in experiences if e is not None])).float().to(device)
        dones = torch.from_numpy(np.vstack([e.done for e in experiences if e is not None]).astype(np.uint8)).float().to(device)
  
        return (states, actions, rewards, next_states, dones)

    def __len__(self):
        """Return the current size of internal memory."""
        return len(self.memory)

In [77]:
class OUNoise:
    """Ornstein-Uhlenbeck process."""

    def __init__(self, size, seed, mu=0., theta=0.15, sigma=0.2):
        """Initialize parameters and noise process."""
        self.mu = mu * np.ones(size)
        self.theta = theta
        self.sigma = sigma
        self.seed = random.seed(seed)
        self.reset()

    def reset(self):
        """Reset the internal state (= noise) to mean (mu)."""
        self.state = copy.copy(self.mu)

    def sample(self):
        """Update internal state and return it as a noise sample."""
        x = self.state
        dx = self.theta * (self.mu - x) + self.sigma * np.array([random.random() for i in range(len(x))])
        self.state = x + dx
        return self.state

In [78]:
from collections import namedtuple, deque
import random
import copy

class DDPGAgent():
    """Interacts with and learns from the environment."""

    def __init__(self, device, hyperparams, actor, critic, state_size, action_size, seed):
        """Initialize an DDPG Agent object.
        
        Params
        ======
            state_size (int): dimension of each state
            action_size (int): dimension of each action
            seed (int): random seed
        """
        self.hyperparams = hyperparams
        
        self.device = device
           
        self.BUFFER_SIZE = hyperparams["BUFFER_SIZE"]
        self.BATCH_SIZE = hyperparams["BATCH_SIZE"]
        self.GAMMA = hyperparams["GAMMA"]
        self.TAU = hyperparams["TAU"]
        self.LR = hyperparams["LR"]
        
        self.LEARN_EVERY = hyperparams["LEARN_EVERY"]
        self.LEARN_ITERATIONS = hyperparams["LEARN_ITERATIONS"]
        
        self.state_size = state_size
        
        self.action_size = action_size
        self.seed = random.seed(seed)

        # Actor
        self.actor = actor(state_size, action_size, seed).to(self.device)
        self.actor_target = actor(state_size, action_size, seed).to(self.device)
        self.actor_optimizer = optim.Adam(self.actor.parameters(), lr=self.LR)
        self.actor.eval()
       
         # Critic
        self.critic = critic(state_size, action_size, seed).to(self.device)
        self.critic_target = critic(state_size, action_size, seed).to(self.device)
        self.critic_optimizer = optim.Adam(self.critic.parameters(), lr=self.LR)
        self.critic.train()
        
        # Deep Q-Learning Algorithm: Initialize replay memory D with N = BUFFER_SIZE
        self.memory = ReplayBuffer(action_size, self.BUFFER_SIZE, self.BATCH_SIZE, seed)
       
        # Initialize time step (for updating every UPDATE_EVERY steps)
        self.t_step = 0
        
         # Noise process
        self.noise = OUNoise(action_size, seed)
            
    def step(self, state, action, reward, next_state, done):
        # Save experience tuple (S, A, R, S~) in replay memory D
        
        for s, a, r, n, d in zip(state, action, reward, next_state, done):
            self.memory.add(s, a, r, n, d)
        
        # Learn every UPDATE_EVERY time steps.
        self.t_step = (self.t_step + 1) % self.LEARN_EVERY
        if self.t_step == 0:
            # If enough samples are available in the memory, run learning cycle
            if len(self.memory) > self.BATCH_SIZE:
                # run learning cycle LEARN_ITERATIONS times 
                for _ in range(self.LEARN_ITERATIONS):
                    # Obtain random minibatch of tuples (S, A, R, S~) from replay memory D
                    experiences = self.memory.sample()
                    self.learn(experiences, self.GAMMA)
                    
                    # update target networks using soft update
                    self.soft_update(self.actor, self.actor_target, self.TAU)   
                    self.soft_update(self.critic, self.critic_target, self.TAU)   
    
    def act(self, states, add_noise=True):
        """Returns actions for given state as per current policy.
        
        Params
        ======
            state (array_like): current state
            eps (float): epsilon, for epsilon-greedy action selection
        """
        
        states = torch.from_numpy(states).float().to(self.device)
        self.actor.eval()
        
        with torch.no_grad():
            action_values = self.actor(states).cpu().data.numpy()
        self.actor.train()
        
        # add noise to the actions to favor exploartion
        if add_noise:
            action_values += self.noise.sample()
        
        # make sure that resulting value after adding noise is still in [-1, 1] interval
        return np.clip(action_values, -1.0, 1.0)

    def learn(self, experiences, gamma):
        """Update value parameters using given batch of experience tuples.

        Params
        ======
            experiences (Tuple[torch.Variable]): tuple of (s, a, r, s', done) tuples 
            gamma (float): discount factor
        """
        
        # Use random minibatch of tuples(S[j], A[j], R[j], S[j+1]) from replay memory D
        states, actions, rewards, next_states, dones = experiences
        
        with torch.no_grad():            
            # get continious actions vector according to the target policy network (Actor)
            # for the next states
            actions_next = self.actor_target(next_states)
            
            # evaluate q values for the actions proposed by policy network for the next states
            # Q(states_next, actions_next) by target Q network (Critic)
            Q_states_next_actions_next = self.critic_target(next_states,  actions_next)
                       
            # calculate target value
            # y[j]= r[j] if episode terminates (done = 1)
            y = rewards + gamma * Q_states_next_actions_next * (1.0 - dones)
        
        # get expected Q value 
        Q_expected = self.critic(states, actions)
        
        # critic loss is normal MSE loss, we need to minimize
        # difference between actual Q values and target Q values for specific state, action pairs
        critic_loss = F.mse_loss(Q_expected, y)
        
        # Minimize the loss
        self.critic_optimizer.zero_grad()
        critic_loss.backward()
        torch.nn.utils.clip_grad_norm(self.critic.parameters(), 1.0)
        self.critic_optimizer.step()
        
        # predict new actions according to current policy network
        predicted_actions = self.actor(states)
        
        # we need to maximize Q values calculated by critic based on predicted actions,
        # so we need to perform gradient ascent (minus sign)
        actor_loss = -self.critic(states, predicted_actions).mean()
        
        # backpropagation
        self.actor_optimizer.zero_grad()
        actor_loss.backward()
        self.actor_optimizer.step()
        
        self.noise.reset()
        
    def soft_update(self, local_model, target_model, tau):
        """Soft update model parameters.
        θ_target = τ*θ_local + (1 - τ)*θ_target

        Params
        ======
            local_model (PyTorch model): weights will be copied from
            target_model (PyTorch model): weights will be copied to
            tau (float): interpolation parameter 
        """
        for target_param, local_param in zip(target_model.parameters(), local_model.parameters()):
            target_param.data.copy_(tau*local_param.data + (1.0-tau)*target_param.data)

In [79]:
def train(env, agent, n_episodes=100, max_t=1000, model_file_actor = 'actor.pth', model_file_critic = 'critic.pth'):
    """Deep Q-Learning.
    
    Params
    ======
        n_episodes (int): maximum number of training episodes
        max_t (int): maximum number of timesteps per episode
        eps_start (float): starting value of epsilon, for epsilon-greedy action selection
        eps_end (float): minimum value of epsilon
        eps_decay (float): multiplicative factor (per episode) for decreasing epsilon
    """
    all_scores = []                     # list containing scores from each episode
    scores_window = deque(maxlen=100)   # last 100 scores
    scores_window10 = deque(maxlen=10)  # last 10 scores
    max_score = 0
    
    for i_episode in range(1, n_episodes+1):
        
        # Not sure why - sometimes unity environment is not reset after first call.
        # so call reset two times - works like a charm
        states = env.reset(train_mode=False)
        
        # reset scores for the episode
        scores = np.zeros(states.shape[0], dtype=float)
        
        # for time step t <- 1 to T:
        for t in range(max_t):
           
            # Choose action from state S using policy network:
            actions = agent.act(states)
            print(actions[0])
            # Take an action and observe result
            next_states, rewards, dones = env.step(actions)
                
            # Sample and Learn section
            agent.step(states, actions, rewards, next_states, dones)
            
            states = next_states
            scores += rewards
            
            if np.any(dones):
                break 
      
        mean_score = np.mean(scores)
        scores_window10.append(mean_score)
        scores_window.append(mean_score)       # save most recent score
        all_scores.append(mean_score)              # save most recent score
        
        print('Episode {}\t Episode score: {:.3f} \t Mean-10 score: {:.3f} \t Mean-100 score: {:.3f}'
              .format(i_episode, mean_score, np.mean(scores_window10), np.mean(scores_window)))
        if i_episode % 100 == 0:
            print('\rEpisode {}\tAverage Score over 100 episodes: {:.3f}'.format(i_episode, np.mean(scores_window)))
        
            if np.mean(scores_window) > max_score:
                print('Saving model... Episode: {} \tAverage Score: {:.6f}\n'.format(i_episode, np.mean(scores_window)))
                torch.save(agent.actor.state_dict(), model_file_actor)
                torch.save(agent.critic.state_dict(), model_file_critic)
                max_score = np.mean(scores_window)
        
        if np.mean(scores_window) > 30.5:
                print('Solved! Saving model... Episode: {} \tAverage Score: {:.6f}\n'.format(i_episode, np.mean(scores_window)))
                torch.save(agent.actor.state_dict(), model_file_actor)
                torch.save(agent.critic.state_dict(), model_file_critic)
                max_score = np.mean(scores_window)
                break
                
    return all_scores

In [80]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

def plot_scores(scores):
    fig = plt.figure()
    ax = fig.add_subplot(111)
    plt.plot(np.arange(len(scores)), scores)
    plt.ylabel('Score')
    plt.xlabel('Episode #')
    plt.show()
    
def moving_average(data_set, periods=3):
    weights = np.ones(periods) / periods
    return np.convolve(data_set, weights, mode='valid')

def plot_comparison(models,  labels, periods):
    
    fig=plt.figure(figsize=(18, 16), dpi= 80, facecolor='w', edgecolor='k')
    
    for m, l in zip(models, labels):
        moving_m =  moving_average(m, periods)
        plt.plot(moving_m, label=l)
    
    plt.legend(loc='lower right')

    plt.title("DQN Models comparison", fontsize=12, fontweight='bold')
    plt.xlabel("Eisode #")
    plt.ylabel("Score")
    
    plt.show()

In [ ]:
hyperparams = { "BUFFER_SIZE" : int(1e6),  # replay buffer size
                "BATCH_SIZE" : 1024,         # minibatch size
                "GAMMA" : 0.99,             # discount factor
                "TAU" : 1e-3,               # for soft update of target parameters
                "LR" : 1e-4,                # learning rate 
                "LEARN_EVERY" : 25,         # how often to update the network
                "LEARN_ITERATIONS" : 10,    # how many iterations needed for each network update
              }

agent = DDPGAgent(device, hyperparams, Actor, Critic, state_size=env.state_size, action_size=env.action_size, seed=12)

scores = train(env, agent, n_episodes=500, max_t=1000, model_file_actor = 'DDPG_actor.pth', model_file_critic = 'DDPG_critic.pth')

np.save("DDPGScores.npy", scores)

[0.13598308 0.09750964]
[0.27006134 0.10472485]
[0.22301912 0.17979294]
[0.20468241 0.33752236]
[0.2755928  0.37945786]
[0.37768716 0.44030547]
[0.35204092 0.43674606]
[0.2885405 0.5421795]
[0.2965141  0.63700235]
Episode 1	 Episode score: -0.005 	 Mean-10 score: -0.005 	 Mean-100 score: -0.005
[0.3154746 0.6711639]
[0.32838866 0.6090874 ]
[0.2962     0.48525536]
[0.23698494 0.5576637 ]
[0.297254   0.49683085]
[0.47830415 0.5948889 ]
[0.46213785 0.56950396]
[0.40318015 0.55449224]
[0.47225374 0.5236563 ]
[0.5581178  0.43136707]
[0.5015309 0.5138374]
[0.45687047 0.5336969 ]
[0.50429267 0.56310654]
[0.51039684 0.4804983 ]
Episode 2	 Episode score: -0.005 	 Mean-10 score: -0.005 	 Mean-100 score: -0.005
[0.45289767 0.49037066]
[0.8750796 0.5561249]
[0.6536945  0.45840895]
[0.586025   0.45769915]
[0.47912732 0.367195  ]
[0.45128977 0.37410963]
[0.5334557  0.38666484]
[0.45093694 0.45083854]
[0.45096424 0.48347926]
[0.42259067 0.49006906]
[0.4321052 0.5588962]
[0.3963366  0.58071065]
[0.371

[1.         0.56844383]
[0.67853886 0.38731146]
[0.504878   0.32216462]
[0.51694775 0.46517247]
[0.6072465 0.4144528]
[0.6373188  0.39765844]
[0.6934477 0.4623033]
[0.7850936 0.5532074]
[0.8067107 0.5764865]
[0.70326906 0.57537764]
[0.66550255 0.51379174]
[0.6203368  0.51144254]
[0.5631387  0.54205316]
Episode 21	 Episode score: -0.005 	 Mean-10 score: -0.005 	 Mean-100 score: -0.005
[0.41935837 0.62755966]
[0.7495132 0.6135926]
[0.5298224 0.461872 ]
[0.4096097  0.53387445]
[0.5130588  0.58835185]
[0.5535096  0.45721218]
[0.57415015 0.3846183 ]
[0.54540277 0.43141773]
[0.48521167 0.40919608]
[0.54602647 0.44306213]
[0.6597567  0.39530924]
[0.59445775 0.51525205]
[0.5863994  0.55439407]
[0.63528746 0.49824223]
Episode 22	 Episode score: -0.005 	 Mean-10 score: -0.005 	 Mean-100 score: -0.005
[0.81368613 0.5918565 ]
[0.9806175  0.47057226]
[0.6581092  0.44400063]
[0.61557615 0.5747407 ]
[0.65593314 0.5242177 ]
[0.58129746 0.4424534 ]
[0.6672531  0.56107205]
[0.7187753 0.5430685]
[0.58374

C:\Users\alexeys\AppData\Local\Continuum\anaconda3\envs\drlnd\lib\site-packages\ipykernel_launcher.py:135: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.


[-0.044824   -0.28392756]
[-0.19616868 -0.15147159]
[-0.00821146 -0.07471952]
[ 0.09544679 -0.05959971]
[ 0.01685594 -0.10857607]
[-0.03882873  0.01475867]
[0.05067563 0.20041233]
[0.19157028 0.31807125]
[0.18428014 0.4083225 ]

When finished, you can close the environment.

In [ ]:
env.close()

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```